In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


*Weighted DC-Kmeans model:*

In [0]:
import numpy as np

class Kmeans(object):
    '''
    In-house implementation of k-means via Lloyd-Max iterations
    This is a research prototype and is not necessarily well-optimized
    '''
    def __init__(self,
                    k,
                    termination='fixed',
                    iters=10,
                    tol=10**-3):
        '''
        Constructor
        INPUT:
        k - # of centroids/clusters
        iters - # of iterations to run
        termination - {'fixed', 'loss', 'centers'}
            if 'fixed' - runs for fixed # of iterations
            if 'loss' - runs until loss converges
            if 'centers' -runs until centers converge
        tol - numeric tolerance to determine convergence
        '''
        # set parameters
        self.k = k
        self.iters = iters
        self.tol = tol
        self.termination = termination
        # initialize placeholder values
        self._init_placeholders()
        self.mass_arr = None

    def run(self, X):
        '''
        Run clustering algorithm
        INPUT:
        X - numpy matrix, n-by-d, each row is a data point
        OUTPUT: (3-tuple)
        centroids - k-by-d matrix of centroids
        assignments - Vector of length n, with datapoint to center assignments
        loss - The loss of the final partition
        '''
        self._set_data(X)
        self._lloyd_iterations()
        return self.centroids, self.assignments, self.loss

    def delete(self, del_idx):
        '''
        Delete point associated with key del_idx
        NOTE: del_idx must be int in {0,n-1}
            After deleting any key other than n-1,
            the (n-1)-th datapoint's key is automatically
            swapped with del_idx to
        '''
        self.data = np.delete(self.data, del_idx, 0)
        self.n = self.n-1
        self._init_placeholders()
        return self.run(self.data)

    def _init_placeholders(self):
        self.loss = np.Infinity
        self.empty_clusters = []
        self.kpp_inits = set()
        self.centroids = None
        self.assignments = None
        self.model = None

    def _set_data(self, X):
        self.data = X
        self.n, self.d = X.shape

    def _lloyd_iterations(self):
        self._init_centroids()
        for _ in range(self.iters):
            loss_prev = self.loss
            centers_prev = self.model
            self._assign_clusters()
            self._assign_centroids()
            prev = loss_prev if self.termination == 'loss' else centers_prev
            if self._check_termination(prev):
                break
            
    def _check_termination(self, prev):
        if self.termination == 'loss':
            return (1 - self.loss/prev) < self.tol
        elif self.termination == 'center':
            return np.linalg.norm(self.centroids - prev) < self.tol
        else:
            return False

    def _init_centroids(self):
        '''
        Kmeans++ initialization
        Returns vector of initial centroids
        '''
        prob = []
        total_mass = 0
        for m in self.mass_arr:
            total_mass += m
        for m in self.mass_arr:
          prob.append(m / total_mass)
        first_idx = np.random.choice(self.n, p = prob)
        self.centroids = self.data[first_idx,:]
        for kk in range(1,self.k):
            P = self._get_selection_prob()
            nxt_idx = np.random.choice(self.n,p=P)
            self.kpp_inits.add(nxt_idx)
            self.centroids = np.vstack([self.centroids,self.data[nxt_idx,:]])

    def _get_selection_prob(self):
        '''
        Outputs vector of selection probabilites
        Equal to Distance^2 to nearest centroid
        '''
        #handle edge case in centroids shape by unsqueezing
        if len(self.centroids.shape) == 1:
            self.centroids = np.expand_dims(self.centroids, axis=0)

        #probability is square distance to closest centroid
        D = np.zeros([self.n])
        for i in range(self.n):
            d = np.linalg.norm(self.data[i,:] - self.centroids, axis=1)
            D[i] = np.min(d)
        P = []
        for i in range(self.n):
            P.append(self.mass_arr[i] * (D[i]**2))
        P = P / sum(P)
        return P  

    def _assign_centroids(self):
        '''
        Computes centroids in Lloyd iterations
        '''
        self.centroids = np.zeros([self.k,self.d])
        c = np.zeros([self.k])
        for i in range(self.n):
            a = self.assignments[i]
            c[a] += self.mass_arr[i]  # weight by mass
            self.centroids[a,:] += self.mass_arr[i] * self.data[i,:]   # weight by mass
            
        for j in range(self.k):
            self.centroids[j,:] = self.centroids[j,:] / c[j]

        for j in self.empty_clusters: 
            self._reinit_cluster(j)
        self.empty_clusters = []
        
    def _assign_clusters(self):
        '''
        Computes clusters in Lloyd iterations
        '''
        assert (self.k, self.d) == self.centroids.shape, "Centers wrong shape"
        self.assignments = np.zeros([self.n]).astype(int)
        self.loss = 0
        for i in range(self.n):
            d = np.linalg.norm(self.data[i,:] - self.centroids, axis=1)
            d1 = np.linalg.norm(self.data[i,:] - self.centroids, axis=1,ord=1)
            self.assignments[i] = int(np.argmin(d))
            self.loss += self.mass_arr[i] * (np.min(d)**2)  # weight by mass
        self.loss = self.loss / sum(self.mass_arr)
        self.empty_clusters = self._check_4_empty_clusters()

    def _check_4_empty_clusters(self):
        empty_clusters = []
        for kappa in range(self.k):
            if len(np.where(self.assignments == kappa)[0]) == 0:
                empty_clusters.append(kappa)
        return empty_clusters

    def _reinit_cluster(self, j):
        '''
        Gets a failed centroid with idx j (empty cluster)
        Should replace with new k++ init centroid
        in:
            j is idx for centroid, 0 <= j <= n
        out:
            j_prime is idx for next centroid
        side-effects:
            centroids are update to reflect j -> j'
        '''
        P = self._get_selection_prob()
        j_prime = np.random.choice(self.n,p=P)
        self.kpp_inits.add(j_prime)
        self.centroids[j,:] = self.data[j_prime,:]
        return j_prime


class DCnode(Kmeans):
    '''
    A k-means subproblem for the divide-and-conquer tree
    in DC-k-means algorithm
    '''
    def __init__(self, k, iters):
        Kmeans.__init__(self, k, iters=iters)
        self.children = []
        self.parent = None
        self.time = 0
        self.loss = 0
        self.node_data = set()
        self.node_mass = dict() #mass dict
        self.mass_arr = []
        self.data_prop = set()

    def _run_node(self, X):
        self._set_node_data(X)
        self._lloyd_iterations()

    def _set_node_data(self, X):
        #print("X", X.shape)
        self.data = X[list(self.node_data)]
        self.mass_arr = []
        for d in self.data:
          self.mass_arr.append(self.node_mass[tuple(d)])
        self._set_data(self.data)

class WeightedDCKmeans():
    def __init__(self, ks, widths, iters=10):
        '''
        Constructor for quantized k-means solved via Lloyd iterations
        ks - list of k parameter for each layer of DC-tree
        widths - list of width parameter (number of buckets) for each layer
        iters - # of iterations to run 
            (at present, only supports fixed iteration termination)
        '''
        self.ks = ks
        self.widths = widths
        self.dc_tree = self._init_tree(ks,widths,iters)
        self.data_partition_table = dict()
        self.data = dict()
        self.mass = dict()  # For mass
        self.dels = set()
        self.valid_ids = []
        self.d = 0
        self.n = 0
        self.h = len(self.dc_tree)
        for i in range(self.h):
            self.data[i] = None
        # For mass
        for i in range(self.h):
            self.mass[i] = None
            
    def run(self, X, assignments=False):
        '''
        X - numpy matrix, n-by-d, each row is a data point
        assignments (optional) - bool flag, computes assignments and loss
            NOTE: Without assignments flag, this only returns the centroids
        OUTPUT:
        centroids - k-by-d matrix of centroids
            IF assignments FLAG IS SET ALSO RETURNS:
        assignments - Vector of length n, with datapoint to center assignments
        loss - The loss of the final partition
        '''
        self._init_data(X)
        self._partition_data(X)
        self._run()
        if assignments:
            assignment_solver = Kmeans(self.ks[0])
            assignment_solver._set_data(X)
            assignment_solver.centroids = self.centroids
            assignment_solver.mass_arr = []
            for i in range(len(X)):
              assignment_solver.mass_arr.append(1)
            assignment_solver._assign_clusters()
            self.assignments = assignment_solver.assignments
            self.loss = assignment_solver.loss
            return self.centroids, self.assignments, self.loss
        return self.centroids
    
    def delete(self, del_idx):
        idx = self.valid_ids[del_idx]
        self.valid_ids[del_idx] = self.valid_ids.pop()
        self.dels.add(idx)
        node = self.dc_tree[-1][self.data_partition_table[idx]]
        node.node_data.remove(idx)
        del node.node_mass[tuple(self.data[self.h-1][idx])]
        l = self.h-1
        self.n -= 1
        while True:
            node._run_node(self.data[l])
            if node.parent == None:
                self.centroids = node.centroids
                break
            data_prop = list(node.data_prop)
            for c_id in range(len(node.centroids)):
                idx = data_prop[c_id]
                self.data[l][idx] = node.centroids[c_id]
            node = node.parent
            l -= 1

    def _init_data(self,X):
        self.n = len(X)
        self.valid_ids = list(range(self.n))
        self.d = len(X[0])
        data_layer_size = self.n
        for i in range(self.h-1,-1,-1):
            self.data[i] = np.zeros((data_layer_size,self.d))
            self.mass[i] = np.zeros((data_layer_size))
            data_layer_size = self.ks[i]*self.widths[i] 
        
    def _partition_data(self, X):
        self.d = len(X[0])
        num_leaves = len(self.dc_tree[-1])
        for i in range(len(X)):
            leaf_id = np.random.choice(num_leaves)
            leaf = self.dc_tree[-1][leaf_id]
            self.data_partition_table[i] = leaf_id
            leaf.node_data.add(i)
            leaf.node_mass[tuple(X[i])] = 1
            #print("partition: ", tuple(X[i]))
            self.data[self.h-1][i] = X[i]

    def _run(self):
        for l in range(self.h-1,-1,-1):
            c = 0
            for j in range(self.widths[l]):
                subproblem = self.dc_tree[l][j]
                subproblem._run_node(self.data[l])
                if subproblem.parent == None:
                    self.centroids = subproblem.centroids
                else:
                    for c_id in range(len(subproblem.centroids)):
                        subproblem.data_prop.add(c)

                        assignment_solver = Kmeans(self.ks[0])
                        X = self.data[l]
                        data = X[list(subproblem.node_data)]    ###????
                        assignment_solver._set_data(data)
                        assignment_solver.centroids = subproblem.centroids
                        assignment_solver.mass_arr = []
                        for i in range(len(data)):
                          assignment_solver.mass_arr.append(1)
                        assignment_solver._assign_clusters()
                        assignments = assignment_solver.assignments
                        i = 0
                        for assign in assignments:
                          if tuple(subproblem.centroids[assign]) not in subproblem.parent.node_mass:
                            subproblem.parent.node_mass[tuple(subproblem.centroids[assign])] = subproblem.mass_arr[i]
                          else:
                            subproblem.parent.node_mass[tuple(subproblem.centroids[assign])] += subproblem.mass_arr[i]
                          i += 1

                        subproblem.parent.node_data.add(c)
                        self.data[l-1][c] = subproblem.centroids[c_id] 
                        c += 1

    def _init_tree(self, ks, widths, iters):
        tree = [[DCnode(ks[0],iters)]] # root node
        for i in range(1,len(widths)):
            k = ks[i]
            assert widths[i] % widths[i-1] == 0, "Inconsistent widths in tree"
            merge_factor = int(widths[i] / widths[i-1])
            level = []
            for j in range(widths[i-1]):
                parent = tree[i-1][j]
                for _ in range(merge_factor):
                    child = DCnode(k,iters=10)
                    child.parent = parent
                    parent.children.append(child)
                    level.append(child)
            tree.append(level)
        return tree


In [5]:
import math
import numpy as np
import matplotlib.pyplot as plt
from del_eff_kmeans import Kmeans, QKmeans, DCKmeans
import time
import pickle
from sklearn.utils import shuffle
from sklearn.metrics import silhouette_score, normalized_mutual_info_score
from random import sample 

'''
D.2 Datasets
• Celltypes [42] consists of 12,009 single cell RNA sequences from a mixture of 4 cell types: microglial cells, endothelial cells, fibroblasts, and mesenchymal stem cells. The data was retrieved from the Mouse Cell Atlas and consists of 10 feature dimensions, reduced from an original 23,433 dimensions using principal component analysis. Such dimensionality reduction procedures are a common practice in computational biology.
• Postures [35, 34] consists of 74,975 motion capture recordings of users performing 5 different hand postures with unlabeled markers attached to a left-handed glove.
• Covtype [12] consists of 15,120 samples of 52 cartographic variables such as elevation and hillshade shade at various times of day for 7 forest cover types.
• Botnet [56] contains statistics summarizing the traffic between different IP addresses for a commercial IoT device (Danmini Doorbell). We aim to distinguish between benign traffic data (49,548 instances) and 11 classes of malicious traffic data from botnet attacks, for a total of 1,018,298 instances.
• MNIST [51] consists of 60,000 images of isolated, normalized, handwritten digits. The task is to classify each 28×28 image into one of the ten classes.
• Gaussian [#] consists of 5 clusters, each generated from 25-variate Gaussian distribution centered at randomly chosen locations in the unit hypercube. 20,000 samples are taken from each of the 5 clusters, for a total of 100,000 samples. Each Gaussian cluster is spherical with variance of 0.8.
  Gaussian consists of 5 clusters, each generated from 25-variate Gaussian distribution
  centered at randomly chosen locations in the unit hypercube. 20,000 samples are
  taken from each of the 5 clusters, for a total of 100,000 samples. Each Gaussian
  cluster is spherical with variance of 0.8.
'''
'''
Celltype (N = 12,009, D = 10, K = 4), "4celltypes_10pca"
Covtype (N = 15,120, D = 52, K = 7), "covtype_multiclass"
MNIST (N = 60,000, D = 784, K = 10), "mnist"
Postures (N = 74,975, D = 15, K = 5), "postures"
Botnet (N = 1,018,298, D = 115, K = 11), "bot_attack" 
and a synthetic dataset made from a Gaussian mixture model which we call Gaussian (N = 100,000, D = 25, K = 5). 
'''

DATAs = ["4celltypes_10pca", "covtype_multiclass", "postures", "mnist", "bot_attack"]
Ks = [4, 7, 5, 10, 11]
mata_loss = {}
mata_silcoef = {}
mata_nmi = {}
mata_runtime = {}
mata_traintime = {}
for d in DATAs:
  # 3 models, each with 5 values
  mata_loss[d] = [[] for j in range(3)]
  mata_silcoef[d] = [[] for j in range(3)]
  mata_nmi[d] = [[] for j in range(3)]
  mata_runtime[d] = [[] for j in range(3)]
  mata_traintime[d] = [[] for j in range(3)]

def show_clustering(centers,assignments,data):
  colors = ['r','b','g']
  for a in range(10):
    data_a = data[assignments == a]
    plt.scatter(data_a[:,0],data_a[:,1])
    plt.scatter(centers[a,0],centers[a,1],marker='x',color='k')
  plt.show()

def model_specific_stats(model, features, labels, dataset, m, save, num_deletions):  # m is model index, 0, 1, 2

  t0 = time.time()
  centers = 0
  assignments = 0
  loss = 0
  if m == 2: 
    centers, assignments, loss = model.run(features.copy(), assignments=True)
  else:
    centers, assignments, loss = model.run(features.copy())
  t1 = time.time()
  print("train time: ", t1 - t0)
  if save:
    mata_traintime[dataset][m].append(t1 - t0)
  
  # Loss
  print('Clustering loss is: ', loss)
  if save:
    mata_loss[dataset][m].append(loss)

  # Silhouette Coefficients
  sampled_index = sample(range(labels.shape[0]), 10000)
  score = silhouette_score(features[sampled_index], assignments[sampled_index])
  print("silhouette_score for 10000 random samples: ", score)
  if save:
    mata_silcoef[dataset][m].append(score)

  # Normalized Mutual Information
  nmi_score = normalized_mutual_info_score(labels, assignments)
  print("normalized_mutual_info_score: ", nmi_score)
  if save:
    mata_nmi[dataset][m].append(nmi_score)

  # Deletion runtime
  print("Online deletion time for " + str(num_deletions) + " deletions: ")
  t = online_deletion_stream(num_deletions, model)
  if save:
      mata_runtime[dataset][m].append((t + t1 - t0) / num_deletions)
  #print("Amortized Runtime to process" + str(num_deletions) + "deletions is ", (t + t1 - t0) / num_deletions)

def reproduce_result(datasets):
  for i in range(5):
    print("Dataset: " + DATAs[i])
    features = datasets[DATAs[i]][0]
    labels = datasets[DATAs[i]][1]
    k = Ks[i]
    n = datasets[DATAs[i]][1].shape[0]
    d = datasets[DATAs[i]][0].shape[1]

    # k means model
    print("k-means model")
    kmeans = Kmeans(k, termination='loss')
    model_specific_stats(kmeans, features, labels, DATAs[i], 0)
    
    # Q-kmeans model
    print("qkmeans model")
    eps = pow(2, -math.log((n / (k * pow(d, 1.5))), 10) - 3)
    qkmeans = QKmeans(k, eps)
    model_specific_stats(qkmeans, features, labels, DATAs[i], 1)

    # DC-kmeans model
    print("dc kmeans model")
    w = pow(2, math.ceil(math.log(pow(n, 0.3))/math.log(2)))
    print(w)
    dckmeans = DCKmeans([k,k],[1,w])
    model_specific_stats(dckmeans, features, labels, DATAs[i], 2)

def online_deletion_stream(num_dels, model):
  t0 = time.time()
  c = 1
  for _ in range(num_dels):
      dr = np.random.choice(model.n,size=1)[0]
      #print(f'processing deletion request # {c}...')
      model.delete(dr)
      c += 1
  t = time.time()
  print(f'Total time to process {c-1} deletions is {t-t0}')
  return t - t0


def tune_tree_height(datasets):
  for i in set([0, 1, 3]):
    print("Dataset: " + DATAs[i])
    features = datasets[DATAs[i]][0]
    labels = datasets[DATAs[i]][1]
    k = Ks[i]
    n = datasets[DATAs[i]][1].shape[0]
    d = datasets[DATAs[i]][0].shape[1]
    
    dckmeans = DCKmeans([k, k, k, k],[1, 4, 16, 64])
    model_specific_stats(dckmeans, features, labels, DATAs[i], 2, True, 20)

    dckmeans = DCKmeans([k, k, k],[1, 8, 64])
    model_specific_stats(dckmeans, features, labels, DATAs[i], 2, True, 20)

    dckmeans = DCKmeans([k, k],[1, 64])
    model_specific_stats(dckmeans, features, labels, DATAs[i], 2, True, 20)


def tune_tree_buckets(datasets):
    for i in set([0, 1, 3]):
      print("Dataset: " + DATAs[i])
      features = datasets[DATAs[i]][0]
      labels = datasets[DATAs[i]][1]
      k = Ks[i]
      n = datasets[DATAs[i]][1].shape[0]
      d = datasets[DATAs[i]][0].shape[1]
      
      for j in [2, 3, 4, 5, 6, 7, 8, 9]:
        dckmeans = DCKmeans([k, k],[1, 2 ** j])
        model_specific_stats(dckmeans, features, labels, DATAs[i], 2, True, 20)

def weighted_tree(datasets):
  for i in set([0, 1, 2, 3, 4]):
      print("Dataset: " + DATAs[i])
      features = datasets[DATAs[i]][0]
      labels = datasets[DATAs[i]][1]
      k = Ks[i]
      n = datasets[DATAs[i]][1].shape[0]
      d = datasets[DATAs[i]][0].shape[1]
      
      w = pow(2, math.ceil(math.log(pow(n, 0.3))/math.log(2)))
      print(w)
      
      for j in range(3):
        print("Weighted")
        w_dckmeans = WeightedDCKmeans([k,k],[1,w])
        model_specific_stats(w_dckmeans, features, labels, DATAs[i], 2, True, 1)
      
      for j in range(3):
        print("non weighted")
        dckmeans = DCKmeans([k,k],[1,w])
        model_specific_stats(dckmeans, features, labels, DATAs[i], 2, True, 1)
        
def weighted_tree2(datasets):
  for i in set([4]):
      print("Dataset: " + DATAs[i])
      features = datasets[DATAs[i]][0]
      labels = datasets[DATAs[i]][1]
      k = Ks[i]
      n = datasets[DATAs[i]][1].shape[0]
      d = datasets[DATAs[i]][0].shape[1]
      
      w = pow(2, math.ceil(math.log(pow(n, 0.3))/math.log(2)))
      print(w)
      '''
      for j in range(3):
        print("Weighted")
        w_dckmeans = WeightedDCKmeans([k,k],[1,w])
        model_specific_stats(w_dckmeans, features, labels, DATAs[i], 2, True, 1)
      '''
      for j in range(3):
        print("non weighted")
        dckmeans = DCKmeans([k,k],[1,w])
        model_specific_stats(dckmeans, features, labels, DATAs[i], 2, True, 1)
        
if __name__ == "__main__":
  
  with open("/content/drive/My Drive/DeleteEfficient/kmeans_data_deletion_NeurIPS19_datasets_scaled.p", mode='rb') as f:
    datasets = pickle.load(f)

  tune_tree_height(datasets)
  print("loss: ")
  for d in DATAs:
    print(d, mata_loss[d])
  print("\nsilhouette coef: ")
  for d in DATAs:
    print(d, mata_silcoef[d])
  print("\nnmi: ")
  for d in DATAs:
    print(d, mata_nmi[d])
  print("\nruntime: ")
  for d in DATAs:
    print(d, mata_runtime[d])
  '''  
  for i in range(5):
    reproduce_result(datasets)
  
  print("loss: ")
  for d in DATAs:
    print(d, mata_loss[d])
  print("\nsilhouette coef: ")
  for d in DATAs:
    print(d, mata_silcoef[d])
  print("\nnmi: ")
  for d in DATAs:
    print(d, mata_nmi[d])
  print("\nruntime: ")
  for d in DATAs:
    print(d, mata_runtime[d])
  '''

  '''
  tune_tree_buckets(datasets)
  print("loss: ")
  for d in DATAs:
    print(d, mata_loss[d])
  print("\nsilhouette coef: ")
  for d in DATAs:
    print(d, mata_silcoef[d])
  print("\nnmi: ")
  for d in DATAs:
    print(d, mata_nmi[d])
  print("\nruntime: ")
  for d in DATAs:
    print(d, mata_runtime[d])
  
  for i in range(len(DATAs)):
    n = datasets[DATAs[i]][1].shape[0]
    w = pow(2, math.ceil(math.log(pow(n, 0.3))/math.log(2)))
    print(w)
  '''

Dataset: 4celltypes_10pca
train time:  4.541198968887329
Clustering loss is:  0.07043462345989038
silhouette_score for 10000 random samples:  0.6466446618121309
normalized_mutual_info_score:  0.052616379980917234
Online deletion time for 20 deletions: 


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


Total time to process 20 deletions is 1.4788517951965332
train time:  3.947449207305908
Clustering loss is:  0.07708804663717818
silhouette_score for 10000 random samples:  0.6422856006930893
normalized_mutual_info_score:  0.038441320597657966
Online deletion time for 20 deletions: 


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


Total time to process 20 deletions is 1.5109143257141113
train time:  3.938832998275757
Clustering loss is:  0.02371198958964685
silhouette_score for 10000 random samples:  0.5316485837472027
normalized_mutual_info_score:  0.20987959737194256
Online deletion time for 20 deletions: 


...

Total time to process 20 deletions is 13.1165771484375
train time:  40.525975942611694
Clustering loss is:  40.09476200474456
silhouette_score for 10000 random samples:  0.06784065529451926
normalized_mutual_info_score:  0.4871695718759179
Online deletion time for 20 deletions: 


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


Total time to process 20 deletions is 13.765998601913452
train time:  40.24881434440613
Clustering loss is:  39.84932643107801
silhouette_score for 10000 random samples:  0.07439360937836823
normalized_mutual_info_score:  0.4993270208776445
Online deletion time for 20 deletions: 


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


Total time to process 20 deletions is 19.649147987365723
loss: 
4celltypes_10pca [[], [], [0.07043462345989038, 0.07708804663717818, 0.02371198958964685]]
covtype_multiclass [[], [], [1.105109404189058, 1.139726344532491, 0.9865279847263123]]
postures [[], [], []]
mnist [[], [], [40.31207756539614, 40.09476200474456, 39.84932643107801]]
bot_attack [[], [], []]

silhouette coef: 
4celltypes_10pca [[], [], [0.6466446618121309, 0.6422856006930893, 0.5316485837472027]]
covtype_multiclass [[], [], [0.23095893986408836, 0.23200795869542062, 0.19559611248273953]]
postures [[], [], []]
mnist [[], [], [0.07241111107344628, 0.06784065529451926, 0.07439360937836823]]
bot_attack [[], [], []]

nmi: 
4celltypes_10pca [[], [], [0.052616379980917234, 0.038441320597657966, 0.20987959737194256]]
covtype_multiclass [[], [], [0.35364759747669544, 0.31681786957500724, 0.3402574839860663]]
postures [[], [], []]
mnist [[], [], [0.4855573014135641, 0.4871695718759179, 0.4993270208776445]]
bot_attack [[], [], 

X (12009, 10)
X (12009, 10)
X (12009, 10)
X (12009, 10)
X (12009, 10)
X (12009, 10)
X (12009, 10)
X (12009, 10)
X (12009, 10)
X (90, 10)
X (90, 10)
X (90, 10)
X (30, 10)
train time:  13.523385763168335
Clustering loss is:  0.009875495187645595
silhouette_score for 10000 random samples:  0.3161159931897116
normalized_mutual_info_score:  0.3696796506454962
Online deletion time for 100 deletions: 
processing deletion request # 1...
X (12009, 10)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


X (90, 10)
X (30, 10)
processing deletion request # 2...
X (12009, 10)


KeyError: ignored

In [0]:
import matplotlib.pyplot as plt
x = [2, 3, 4, 5, 6, 7, 8, 9]
nmi1 = [0.3566097971284424, 0.31829612599551427, 0.26160267063730136, 0.2305281490495208, 0.19757822958459795, 0.2033565054065221, 0.29498712519888504, 0.28831689847928715]
nmi2 = [0.3425311624096031, 0.3094292442236454, 0.35199981877611436, 0.3318195994281116, 0.3293841491553029, 0.33256364404326855, 0.32973415740883605, 0.3547205599550721]
nmi3 = [0.48118463715576776, 0.43701346398934654, 0.4907510934535729, 0.49129394972671414, 0.4969504988136086, 0.473456792609371, 0.4853471975095277, 0.4639135028397989]
plt.xlabel('Tree width (number of nodes in the second layer)')
plt.ylabel('Normalized Mutual Information score')
plt.show()

plt.xlabel('Tree width (number of nodes in the second layer)')
plt.ylabel('Silhouette Coefficient')
plt.show()

plt.xlabel('Tree width (number of nodes in the second layer)')
plt.ylabel('Loss')
plt.show()

plt.xlabel('Tree width (number of nodes in the second layer)')
plt.ylabel('Amortized Runtime')
plt.show()


'''
loss: 
4celltypes_10pca [[], [], [0.018650796115511764, 0.021989759573573724, 0.02123363641761834, 0.027181128695309625, 0.025302743263513207, 0.022640554885324852, 0.02343571199078768, 0.02120072788214959]]
covtype_multiclass [[], [], [1.0093872899568472, 1.0152784877894863, 0.9924148811850896, 0.9681774327169707, 1.039034620146096, 1.0359270546129151, 0.9939269616770611, 1.0239888286047558]]
postures [[], [], []]
mnist [[], [], [39.6887819207228, 40.85613631974319, 40.033193525386665, 39.61676658453061, 39.824881151451414, 40.029429842563395, 39.98630161691774, 40.603515379831684]]
bot_attack [[], [], []]

silhouette coef: 
4celltypes_10pca [[], [], [0.38458809948056094, 0.3255203098801613, 0.4061978235197673, 0.4920635885266285, 0.5391688081505716, 0.5333195615716323, 0.4128338180162358, 0.41701331060616875]]
covtype_multiclass [[], [], [0.2192693277128411, 0.21265828300708067, 0.19295527681699381, 0.28230270768701393, 0.2193945966633595, 0.2204125519130182, 0.188395444095423, 0.23985626611193028]]
postures [[], [], []]
mnist [[], [], [0.06841322262967865, 0.05969255745734885, 0.06913850133189553, 0.05874127371162567, 0.06744032478794254, 0.07063882723256279, 0.075351388420552, 0.07440175793721708]]
bot_attack [[], [], []]

nmi: 
4celltypes_10pca [[], [], [0.3566097971284424, 0.31829612599551427, 0.26160267063730136, 0.2305281490495208, 0.19757822958459795, 0.2033565054065221, 0.29498712519888504, 0.28831689847928715]]
covtype_multiclass [[], [], [0.3425311624096031, 0.3094292442236454, 0.35199981877611436, 0.3318195994281116, 0.3293841491553029, 0.33256364404326855, 0.32973415740883605, 0.3547205599550721]]
postures [[], [], []]
mnist [[], [], [0.48118463715576776, 0.43701346398934654, 0.4907510934535729, 0.49129394972671414, 0.4969504988136086, 0.473456792609371, 0.4853471975095277, 0.4639135028397989]]
bot_attack [[], [], []]

runtime: 
4celltypes_10pca [[], [], [13.825237035751343, 6.947296857833862, 3.735635995864868, 2.4677200317382812, 2.105858087539673, 3.104292154312134, 5.404259920120239, 10.086616039276123]]
covtype_multiclass [[], [], [21.948658227920532, 10.81125783920288, 5.813812017440796, 4.123564958572388, 3.8815248012542725, 5.878916263580322, 10.245152950286865, 20.703763961791992]]
postures [[], [], []]
mnist [[], [], [176.21602416038513, 85.9961142539978, 44.67074203491211, 26.727442026138306, 18.86534309387207, 20.994234085083008, 34.139708280563354, 63.156938791275024]]
bot_attack [[], [], []]

traintime: 
4celltypes_10pca [[], [], [3.0344059467315674, 3.032461166381836, 2.9939041137695312, 3.106005907058716, 3.138749837875366, 3.425178050994873, 3.518235206604004, 4.029762029647827]]
covtype_multiclass [[], [], [4.795390844345093, 4.78167200088501, 4.499297142028809, 4.909643173217773, 5.128259897232056, 4.680608749389648, 5.149761199951172, 5.842344045639038]]
postures [[], [], []]
mnist [[], [], [37.85356378555298, 36.322457790374756, 37.79789113998413, 36.96315813064575, 39.328386068344116, 39.643174171447754, 39.99007725715637, 42.39094400405884]]
bot_attack [[], [], []]

'''

'''
x = np.arange(10)

plt.plot(x, x)
plt.plot(x, 2 * x)
plt.plot(x, 3 * x)
plt.plot(x, 4 * x)

plt.legend(['y = x', 'y = 2x', 'y = 3x', 'y = 4x'], loc='upper left')

plt.show()

'''
